In [2]:
import pandas as pd
import pandahouse as ph

import requests
from urllib.parse import urlencode

## Задание 2. SQL

### 2.1 Очень усердные ученики.

Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной".

Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин.
Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов за март 2020 года.

NB! Под усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.

Дана таблица `peas`:
<br>
<br>`st_id` (int) - ID ученика
<br>`timest` (timestamp) - Время решения карточки
<br>`correct` (bool) - Правильно ли решена горошина?
<br>`subject` (text) - Дисциплина, в которой находится горошина

In [3]:
# подключаемся к БД

connection = dict(database='default',
                  host='https://clickhouse.lab.karpov.courses',
                  user='student',
                  password='dpo_python_2020')

In [4]:
query_1 = """
SELECT COUNT(st_id) as num_of_studs
FROM
    (SELECT DISTINCT st_id 
    FROM peas
    WHERE toStartOfMonth(timest) == '2020-03-01'
    GROUP BY 
        st_id
    HAVING  SUM(correct) >= 20)
"""
hardworking_studs = ph.read_clickhouse(query=query_1, connection=connection)
hardworking_studs

,num_of_studs
0,0


Таких учеников не оказалось.

### 2.2 Оптимизация воронки.

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

Даны три таблицы:
1. `peas`

<br>`st_id` (int) - ID ученика
<br>`timest` (timestamp) - Время решения карточки
<br>`correct` (bool) - Правильно ли решена горошина?
<br>`subject` (text) - Дисциплина, в которой находится горошина
<br>
<br>

2. `studs`

<br>`st_id` (int) - ID ученика
<br>`test_grp` (text) - Метка ученика в данном эксперименте
<br>
<br>

3. `final_project_check`

<br>`st_id` (int) - ID ученика
<br>`sale_time` (timestamp) - Время покупки
<br>`money` (int) - Цена, по которой приобрели данный курс
<br>`subject` (text) - Дисциплина, на которую приобрели полный доступ

Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:

* ARPU 
* ARPAU 
* CR в покупку 
* СR активного пользователя в покупку 
* CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике

In [5]:
query_metrics = """

WITH unique_users AS (
    SELECT DISTINCT st_id, test_grp
    FROM studs
    GROUP BY st_id, test_grp
    ),
    all_data AS (
    SELECT a.st_id AS st_id, 
        test_grp, 
        all_money, 
        math_money,
        all_tasks,
        all_math_tasks
    FROM unique_users AS a
    LEFT JOIN (SELECT st_id, 
                    SUM(money) AS all_money, 
                    sumIf(money, subject=='Math') AS math_money 
                FROM final_project_check 
                GROUP BY st_id) AS b 
    ON a.st_id == b.st_id
    LEFT JOIN (SELECT st_id, 
                    COUNT(correct) AS all_tasks,  
                    countIf(correct, subject=='Math') AS all_math_tasks 
                FROM peas 
                GROUP BY st_id) AS c 
    ON a.st_id == c.st_id
    )
    
SELECT
test_grp,
SUM(all_money) / COUNT(st_id) as ARPU,
sumIf(all_money, all_tasks > 0) / countIf(st_id, all_tasks > 0) as ARPAU,
countIf(st_id, all_money > 0) / COUNT(st_id) as CR_purchase,
countIf(st_id, all_money > 0 and all_tasks > 0) / countIf(st_id, all_tasks > 0) as CR_purchase_active,
countIf(st_id, math_money > 0 and all_math_tasks > 0) / countIf(st_id, all_math_tasks > 0) as CR_math

FROM all_data
GROUP BY test_grp
""" 

metrics = ph.read_clickhouse(query=query_metrics, connection=connection)
metrics

,test_grp,ARPU,ARPAU,CR_purchase,CR_purchase_active,CR_math
0,control,4540.983607,8393.939394,0.049180,0.090909,0.056604
1,pilot,11508.474576,22832.167832,0.108475,0.209790,0.088889
